In [ ]:
import json, sys, os, glob
import pandas as pd
from datetime import date, time, datetime, timedelta


In [ ]:
symbol = "BTCUSDT"
base_tf = "1d"

In [ ]:
#Put csv data into pandas dataframe
folder_path = f"./data/spot/monthly/klines/{symbol}/{base_tf}/2017-07-01_2022-04-01/"
col_names = ['ts', 'open', 'high', 'low', 'close', 'volume', 'close_ts', 'asset_volume', 'num_trades', 'i','d','k']
all_files = glob.glob(folder_path + "*.csv")
df_list = (pd.read_csv(f, names=col_names, usecols=[0, 1, 2, 3, 4, 5], index_col=0) for f in all_files)
df = pd.concat(df_list)

#Convert epoch int into actual datetime
df.index = df.index.map(lambda x: datetime.utcfromtimestamp(int(x)/1000))
